In [0]:
import yaml
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col

# 1. Konfiguracja Środowiska (jak w notebooku Bronze)
try:
    # Zakładamy, że dbutils.widgets.get() jest dostępne w Databricks
    ENV = dbutils.widgets.get("env_name") 
except Exception:
    ENV = 'TEST' 
    
# 2. Wczytanie Konfiguracji
try:
    # Upewnij się, że ścieżka do config.yaml jest poprawna dla tego notebooka!
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("BŁĄD: Plik 'config.yaml' nie został znaleziony! Sprawdź ścieżkę.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Nie znaleziono konfiguracji dla środowiska: {ENV} w pliku YAML.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']

# Użyj tej samej bazowej ścieżki wyjściowej, co w notebooku Bronze
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_delta_lake_bronze/"
print(f"Ścieżka do warstwy Bronze: {base_output_directory}")

In [0]:
# Ścieżka do katalogu nadrzędnego zawierającego podkatalogi Delta/Parquet
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_delta_lake_bronze/"

try:
    # Kluczowa zmiana: uzywamy formatu "parquet" zamiast "delta"
    df_bronze = (
        spark.read.format("parquet")
        .load(base_output_directory)
        .withColumn("Date", col("Date").cast("date"))
    )
    
    print(f"Pomyślnie wczytano dane. Liczba wierszy: {df_bronze.count()}")
    df_bronze.printSchema()
    
    # Zauważ, że kolumna 'Ticket' zostanie poprawnie wczytana jako kolumna partycji
    df_bronze.select("Ticket", "Date", "Close").show(5)

except Exception as e:
    print(f"BŁĄD: {e}")